In [129]:
## notebook for testing & parsing quickfs api

# todo:  
# [] metrics
#     [] margins
#     [] 10 year, 5 year growth rates
#     [] 10 year, 5 year return metrics
#     [] fcf
#     [] yields
# [] analysis functions
#     [] dcf
#     [] company test (shareholders equity growth /r&d ect )
# [] proper python project structure 
# [] write script ingest data 
# [] hide api key




In [1]:
pip install quickfs

Note: you may need to restart the kernel to use updated packages.


In [1]:
# import libs

from quickfs import QuickFS
import os
from pathlib import Path
import json
import pandas as pd

In [21]:
# load the key from the enviroment variables
api_key = 'c88fedbb6174ed4d87f7972f4cb73325d7beb7c9'

# example functionality
client = QuickFS(api_key)

# # Request reference data for the supported companies
# resp = client.get_api_metadata()
# resp = client.get_supported_companies(country='US', exchange='NYSE')

# Usage history
resp = client.get_usage()

# # Metric Metadata
# resp = client.get_available_metrics()

# This property contains the raw form of the request object.
resp



{'quota': {'used': 623, 'remaining': -123, 'resets': '2022-10-17T23:00:00Z'}}

In [3]:
# list of minimum metrics I would need to do useful analysis:

# Income statement

income = ['revenue', 'cogs', 'gross_profit', 'sga', 'rnd', 'operating_income', 'ebit','income_tax', 'interest_expense','net_income', 'shares_diluted']

# balance sheet

balance = ['cash_and_equiv','receivables', 'inventories',
 'total_current_assets','equity_and_other_investments','ppe_net',
 'intangible_assets', 'goodwill','total_assets', 'accounts_payable',
 'tax_payable', 'st_debt', 'total_current_liabilities', 'lt_debt',
 'noncurrent_capital_leases',
 'total_liabilities',
 'retained_earnings',
 'total_equity']

# cashflow statement
cashflow = ['cf_cfo', 'cfi_ppe_net', 'cfi_acquisitions_net', 'cfi_investment_net', 'cfi_intangibles_net','cff_common_stock_net', 'cff_debt_net', 'cff_dividend_paid', 'cf_cff']

# price info
price =  ['market_cap', 'period_end_price', 'name', 'period_end_date']

metrics = income + balance + cashflow + price


In [20]:
# companies per day
n_metrics = len(metrics )
print('number of metrics:',n_metrics, '\ncompanies to download per day:', 500 // n_metrics)


number of metrics: 42 
companies to download per day: 11


In [15]:
# example of 20 year data batch request
data = client.get_data_batch(companies=['KO:US', 'PEP:US'], metrics=['revenue','cogs'], period="FY-20:FY")


In [6]:
# load tickers from csv

ticker_path = Path(Path().absolute().parent, 'data', 'tickers.csv')
ticker_path

tickers = pd.read_csv(ticker_path).ticker.values
tickers
# [x for x in p.iterdir() if x.is_dir()]

array(['ASC:LN', 'BOO:LN', 'AMZN:US', 'COST:US', 'ZLNDY:US'], dtype=object)

In [19]:
client.get_data_batch(companies=tickers, metrics=metrics, period="FY-20:")


In [18]:
data

In [9]:
# # example of data load and parse process


import json
import pandas as pd

type(data)

# j_data = json.loads(data)
df = pd.DataFrame.from_dict(data, orient='columns')

tdf = df
# 

In [14]:
len(data)

TypeError: object of type 'NoneType' has no len()

In [ ]:
metrics.remove('name')
tdf = tdf.drop(columns='name')
tdf = tdf.explode(metrics)

f_path = Path(Path().absolute().parent, 'data', 'financials.csv')
tdf.to_csv(f_path)

tdf.info()